## Libraries and Imports

In [1]:
from datasets import load_dataset
import random
import cv2
import numpy as np
import sklearn 
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\Marcus\.conda\envs\VIP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## global variables



In [2]:
##
SEED = 39 # random seed for reproducibility. seed =36,37 and 39 are used in the rapport
    # used to select random labels
    # used to split data into train and test
    # used to sample descriptors for k-means training
    # used to set random seed in k-means

num_of_labels = 20 # number of labels to use from the dataset
k = 500  # number of clusters / visual words for k-means clustering
num_samples = 10000  # number of descriptors to sample for k-means training

## Import data

In [3]:
ds_dict = load_dataset("flwrlabs/caltech101")

random.seed(SEED)

ds = ds_dict["train"]
class_names = ds.features["label"].names

# sample 5 random label IDs
num_classes = len(class_names)
selected_label_ids = random.sample(range(num_classes), num_of_labels) 

# filter dataset
ds = ds.filter(lambda x: x["label"] in selected_label_ids)

# We map the ids to class names for printing so we know which labels were selected
selected_labels = [class_names[i] for i in selected_label_ids]
print("Selected labels:", selected_labels)


Selected labels: ['cup', 'emu', 'ibis', 'ant', 'crocodile', 'dollar_bill', 'inline_skate', 'umbrella', 'pizza', 'accordion', 'sea_horse', 'elephant', 'starfish', 'hawksbill', 'sunflower', 'crab', 'faces_easy', 'brain', 'headphone', 'airplanes']


## Split data

In [4]:
ds_split = ds.train_test_split(test_size=0.5, seed=SEED) # define split, it puts the remaining data in "train"

train_ds = ds_split["train"] # training set #50% of data
test_ds  = ds_split["test"] # test set #50% of data

# 1 Codebook generation

## Extract features

In [5]:
sift = cv2.SIFT_create() #SIFT feature extractor, create the SIFT object once

def extract_features(pil_image):

    # Convert PIL image to numpy array
    image = np.array(pil_image)

    # Convert to grayscale
    if image.ndim == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    #extract_features
    keypoints, descriptors = sift.detectAndCompute(image, None)

    # Handle images with no keypoints
    if descriptors is None:
        return np.empty((0, 128), dtype=np.float32)

    return keypoints ,descriptors

In [6]:
train_features = []
train_labels   = []
for sample in train_ds:
    keypoints, descriptors = extract_features(sample["image"])
    train_features.append(descriptors)
    train_labels.append(sample["label"])


test_features = []
test_labels   = []
for sample in test_ds:
    keypoints, descriptors = extract_features(sample["image"])
    test_features.append(descriptors)
    test_labels.append(sample["label"])

# Convert lists to matrix
train_feature_matrix = np.vstack(train_features) # use for training k-means
test_feature_matrix = np.vstack(test_features)

train_feature_matrix.shape, test_feature_matrix.shape #check shapes

((524786, 128), (514161, 128))

The shape above tells us that we currently have 87407 (rows) features represented as vectors of length 128. Each feature describes a patch in a given image in the dataset. A feature could fx. be a pattern, blob, corner, edge or something else.

### Reduce number of features (maybe not neccesary)

In [7]:
#Sample some of the descriptors, to reduce size for k-means

num_dec = train_feature_matrix.shape[0]
sample_size = min(num_samples, num_dec) # so we dont sample more than we have

rng = np.random.default_rng(seed=SEED)  # set random seed
sample_indices = rng.choice(num_dec, size=sample_size, replace=False) # get random indices



train_sampled_descriptors = train_feature_matrix[sample_indices] # get the randomly sampled descriptors
train_sampled_descriptors.shape #check shapes again to make sure sampling worked

(10000, 128)

## K mens clustering

In [8]:

KMeans_model = sklearn.cluster.KMeans(n_clusters=k, random_state=SEED, n_init= 10) # Model definition
KMeans_model.fit(train_sampled_descriptors) # Fit model, which means to train the k-means clustering
codebook = KMeans_model.cluster_centers_  # The cluster centers are our visual words (our codebook)
print("Visual words shape:", codebook.shape)


c:\Users\Marcus\.conda\envs\VIP\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


Visual words shape: (500, 128)


## Form Bag of visual words for training data
classify each training descriptor to the
closest cluster centers and form the bag of words (BoW) for each image in the
image training set. For each image, we find out which words appears, and how many times it appears. Remember that the Bag of visual words, IS the word histogram

In [9]:

# Which visual word (cluster) does each descriptor belong to?
train_word_ids = []

for descriptors in train_features:   # one image at a time
    word_ids = KMeans_model.predict(descriptors)
    train_word_ids.append(word_ids)

# Now we can build the bag-of-words histograms for each image, i.e. 
#count how many times each visual word appears in the image
train_bow = []
for word_ids in train_word_ids:
    hist = np.bincount(word_ids, minlength=k)
    train_bow.append(hist)

train_bow = np.array(train_bow)
train_bow.shape  # Check shape

(1194, 500)

# 2 Indexing
For each image in the test set:

* Extract the SIFT descriptors of the feature points in the image, (DONE)
* Project the descriptors onto the codebook, i.e. for each descriptor the find the closest cluster
* Constructs the generated corresponding bag of words, i.e. word histogram

## Form the bag of words for the testing data

In [10]:

# Which visual word (cluster) does each descriptor belong to?
test_word_ids = []

for descriptors in test_features:   # one image at a time
    word_ids = KMeans_model.predict(descriptors)
    test_word_ids.append(word_ids)
# Now we can build the bag-of-words histograms for each image, i.e. 
#count how many times each visual word appears in the image
test_bow = []
for word_ids in test_word_ids:
    hist = np.bincount(word_ids, minlength=k)
    test_bow.append(hist)

test_bow = np.array(test_bow)
test_bow.shape  # Check shape

(1194, 500)

## Construct table
Per row the the table contains:
* file name
* true category
* training or test set
* bag of words/word histogram

In [11]:
# Create training DataFrame
train_rows = []

for i, sample in enumerate(train_ds):
    row = {
        "filename": sample.get("filename", f"train_{i}"),
        "label": train_labels[i],
        "split": "train",
        "bow": train_bow[i]
    }
    train_rows.append(row)

train_table = pd.DataFrame(train_rows)

# Create test DataFrame
test_rows = []

for i, sample in enumerate(test_ds):
    row = {
        "filename": sample.get("filename", f"test_{i}"),
        "label": test_labels[i],
        "split": "test",
        "bow": test_bow[i]
    }
    test_rows.append(row)

test_table = pd.DataFrame(test_rows)

# Combine the two :)
Table = pd.concat([train_table, test_table], ignore_index=True)
print("Combined table shape: images x categories", Table.shape)

Combined table shape: images x categories (2388, 4)


# 3 Retrieving

Given an image, how similar is the image to other images

## Common words

In [12]:
## common words, how many words are in common between two bags of words
def common_words(bow1, bow2):
    return np.sum(np.minimum(bow1, bow2))

In [13]:
## Basic retrieval function
# compare a query bow, to every bow in the table

def query_common_words(query_bow, table):

    score =[]
    bow_q = query_bow["bow"]

    for i, row in table.iterrows():
        bow_i = row["bow"]

        common_words_score = common_words(bow_q, bow_i)  # Using common words as similarity

        score.append({
            "filename": row["filename"],
            "label": row["label"],
            "score": common_words_score,
        })
    score_df = pd.DataFrame(score)
    score_df = score_df.sort_values(by="score", ascending=False)
    return score_df


## tf-idf similarity

In [14]:
tf_idf_transformer = TfidfTransformer(norm='l2') # an object that can transform BOW to TF-IDF
tf_idf_train = tf_idf_transformer.fit_transform(train_bow) # fitting the transformer AND  transform the training BOW to TF-IDF

In [15]:
def query_tf_idf(query_bow, train_table, tf_idf_transformer, tf_idf_train):

    bow_q = query_bow["bow"].reshape(1, -1)
    bow_q_tf_idf = tf_idf_transformer.transform(bow_q) # transform query to tf-idf
    tf_idf_db = tf_idf_train[train_table.index]
    scores = cosine_similarity(bow_q_tf_idf, tf_idf_db).flatten()


    results = pd.DataFrame({
        "filename": train_table["filename"].values,
        "label": train_table["label"].values,
        "tf_idf_score": scores
    })

    results = results.sort_values(by="tf_idf_score", ascending=False)

    return results


## Bhattacharyya distance

In [16]:
def bhattacharyya(bow1, bow2):
    epsilon = 1e-10
    # Normalize histograms to get probability distributions
    p = bow1 / (np.sum(bow1) + epsilon) # avoid division by zero +1e-10
    q = bow2 / (np.sum(bow2) + epsilon) #

    # Compute Bhattacharyya coefficient
    bc = np.sum(np.sqrt(p * q))
    
    return bc

In [17]:

def query_bhattacharyya(query_bow, table):

    score =[]
    bow_q = query_bow["bow"]

    for i, row in table.iterrows():
        bow_i = row["bow"]

        bhattacharyya_score = bhattacharyya(bow_q, bow_i)  # Using Bhattacharyya similarity

        score.append({
            "filename": row["filename"],
            "label": row["label"],
            "score": bhattacharyya_score,
        })
    score_df = pd.DataFrame(score)
    score_df = score_df.sort_values(by="score", ascending=False)
    return score_df


# Kullback-Liebler divergence

In [18]:
def kullback_leibler(bow1, bow2):
    epsilon = 1e-10
    # Normalize histograms to get probability distributions
    p = bow1 / (np.sum(bow1) + epsilon) # avoid division by zero +1e-10
    q = bow2 / (np.sum(bow2) + epsilon) #

    # Compute Kullback-Leibler divergence
    kl_div = np.sum(p * np.log(p / (q + epsilon)))
    
    return kl_div

In [19]:
def query_kullback_leibler(query_bow, table):

    score =[]
    bow_q = query_bow["bow"]

    for i, row in table.iterrows():
        bow_i = row["bow"]

        kl_score = kullback_leibler(bow_q, bow_i)  # Using Kullback-Leibler divergence

        score.append({
            "filename": row["filename"],
            "label": row["label"],
            "score": kl_score,
        })
    score_df = pd.DataFrame(score)
    score_df = score_df.sort_values(by="score", ascending= True)  # lower KL divergence is better, because it is a distance
    return score_df


## Experiment metrics

In [20]:
def reciprocal_rank(results, true_label):
    for rank, (_, row) in enumerate(results.iterrows(), start=1):
        if row["label"] == true_label:
            return 1 / rank # if found, return reciprocal rank
    return 0  #if true label not found

In [21]:
def top_k_accuracy(results, true_label, k= 3):
    return true_label in results.head(k)["label"].values

## Experiment 1 - Training data

In [22]:
# get query image
# get similar images from training set using 1/4 retrieval methods
# remove the query image from the results
# compute evaluation metrics

#common words retrieval
rr_common_words_train_list = []
top3_common_words_train_list = []

#TF-IDF retrieval
rr_tf_idf_train_list = []
top3_tf_idf_train_list = []

#Bhattacharyya retrieval
rr_bhattacharyya_train_list = []
top3_bhattacharyya_train_list = []
#Kullback-Leibler retrieval
rr_kullback_leibler_train_list = []
top3_kullback_leibler_train_list = []


for i in range(len(train_table)):  # index of the query image in the train set
    query_row = train_table.iloc[i]  # select the first image in the train set as query
    true_label = query_row["label"] # the true label of the query image

    fixed_query_table = train_table.drop(i)

    results_common_words = query_common_words(query_row, fixed_query_table) # common words retrieval
    results_tf_idf = query_tf_idf(query_row, fixed_query_table, tf_idf_transformer, tf_idf_train) # TF-IDF retrieval
    results_bhattacharyya = query_bhattacharyya(query_row, fixed_query_table) # Bhattacharyya retrieval
    results_kullback_leibler = query_kullback_leibler(query_row, fixed_query_table) # Kullback-Leibler retrieval
    # add other retrieval methods here

    #compute evaluation metrics
    #common words retrieval
    rr_common_words_train_list.append(reciprocal_rank(results_common_words, true_label))
    top3_common_words_train_list.append(top_k_accuracy(results_common_words, true_label, k=3))

    #TF-IDF retrieval
    rr_tf_idf_train_list.append(reciprocal_rank(results_tf_idf, true_label))
    top3_tf_idf_train_list.append(top_k_accuracy(results_tf_idf, true_label, k=3))

    #Bhattacharyya retrieval
    rr_bhattacharyya_train_list.append(reciprocal_rank(results_bhattacharyya, true_label))
    top3_bhattacharyya_train_list.append(top_k_accuracy(results_bhattacharyya, true_label, k=3))

    #Kullback-Leibler retrieval
    rr_kullback_leibler_train_list.append(reciprocal_rank(results_kullback_leibler, true_label))
    top3_kullback_leibler_train_list.append(top_k_accuracy(results_kullback_leibler, true_label, k=3))


# compute mean metrics
mean_rr_common_words = np.mean(rr_common_words_train_list)
mean_top3_common_words = np.mean(top3_common_words_train_list)

mean_rr_tf_idf = np.mean(rr_tf_idf_train_list)
mean_top3_tf_idf = np.mean(top3_tf_idf_train_list)

mean_rr_bhattacharyya = np.mean(rr_bhattacharyya_train_list)
mean_top3_bhattacharyya = np.mean(top3_bhattacharyya_train_list)

mean_rr_kullback_leibler = np.mean(rr_kullback_leibler_train_list)
mean_top3_kullback_leibler = np.mean(top3_kullback_leibler_train_list)



C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(p * np.log(p / (q

In [23]:

# print results
print("Results using Common Words:")
print("Reciprocal Rank:", mean_rr_common_words)
print("Top-3 Accuracy:", mean_top3_common_words)

print("Results using TF-IDF:")
print("Reciprocal Rank:", mean_rr_tf_idf)
print("Top-3 Accuracy:", mean_top3_tf_idf)

print("Results using Bhattacharyya:")
print("Reciprocal Rank:", mean_rr_bhattacharyya)
print("Top-3 Accuracy:", mean_top3_bhattacharyya)

print("Results using Kullback-Leibler:")
print("Reciprocal Rank:", mean_rr_kullback_leibler)
print("Top-3 Accuracy:", mean_top3_kullback_leibler)

Results using Common Words:
Reciprocal Rank: 0.2526551045119876
Top-3 Accuracy: 0.2797319932998325
Results using TF-IDF:
Reciprocal Rank: 0.6277182134977988
Top-3 Accuracy: 0.6934673366834171
Results using Bhattacharyya:
Reciprocal Rank: 0.5687178267460524
Top-3 Accuracy: 0.652428810720268
Results using Kullback-Leibler:
Reciprocal Rank: 0.3755638746809426
Top-3 Accuracy: 0.5108877721943048


## Experiment 2 - Test data

In [24]:
# get query image
# get similar images from training set using 1/4 retrieval methods
# remove the query image from the results
# compute evaluation metrics

#common words retrieval
rr_common_words_test_list = []
top3_common_words_test_list = []

#TF-IDF retrieval
rr_tf_idf_test_list = []
top3_tf_idf_test_list = []

#Bhattacharyya retrieval
rr_bhattacharyya_test_list = []
top3_bhattacharyya_test_list = []
#Kullback-Leibler retrieval
rr_kullback_leibler_test_list = []
top3_kullback_leibler_test_list = []


for i in range(len(test_table)):  # index of the query image in the test set
    query_row = test_table.iloc[i]  # select the first image in the test set as query
    true_label = query_row["label"] # the true label of the query image

    fixed_query_table = train_table # copy of train table

    results_common_words = query_common_words(query_row, fixed_query_table) # common words retrieval
    results_tf_idf = query_tf_idf(query_row, fixed_query_table, tf_idf_transformer, tf_idf_train) # TF-IDF retrieval
    results_bhattacharyya = query_bhattacharyya(query_row, fixed_query_table) # Bhattacharyya retrieval
    results_kullback_leibler = query_kullback_leibler(query_row, fixed_query_table) # Kullback-Leibler retrieval
    # add other retrieval methods here

    #compute evaluation metrics
    #common words retrieval
    rr_common_words_test_list.append(reciprocal_rank(results_common_words, true_label))
    top3_common_words_test_list.append(top_k_accuracy(results_common_words, true_label, k=3))

    #TF-IDF retrieval
    rr_tf_idf_test_list.append(reciprocal_rank(results_tf_idf, true_label))
    top3_tf_idf_test_list.append(top_k_accuracy(results_tf_idf, true_label, k=3))

    #Bhattacharyya retrieval
    rr_bhattacharyya_test_list.append(reciprocal_rank(results_bhattacharyya, true_label))
    top3_bhattacharyya_test_list.append(top_k_accuracy(results_bhattacharyya, true_label, k=3))
    #Kullback-Leibler retrieval
    rr_kullback_leibler_test_list.append(reciprocal_rank(results_kullback_leibler, true_label))
    top3_kullback_leibler_test_list.append(top_k_accuracy(results_kullback_leibler, true_label, k=3))


# compute mean metrics
mean_rr_common_words = np.mean(rr_common_words_test_list)
mean_top3_common_words = np.mean(top3_common_words_test_list)

mean_rr_tf_idf = np.mean(rr_tf_idf_test_list)
mean_top3_tf_idf = np.mean(top3_tf_idf_test_list)

mean_rr_bhattacharyya = np.mean(rr_bhattacharyya_test_list)
mean_top3_bhattacharyya = np.mean(top3_bhattacharyya_test_list)

mean_rr_kullback_leibler = np.mean(rr_kullback_leibler_test_list)
mean_top3_kullback_leibler = np.mean(top3_kullback_leibler_test_list)


C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: divide by zero encountered in log
  kl_div = np.sum(p * np.log(p / (q + epsilon)))
C:\Users\Marcus\AppData\Local\Temp\ipykernel_44656\3331463235.py:8: RuntimeWarning: invalid value encountered in multiply
  kl_div = np.sum(p * np.log(p / (q

In [25]:

# print results
print("Results using Common Words:")
print("Reciprocal Rank:", mean_rr_common_words)
print("Top-3 Accuracy:", mean_top3_common_words)

print("Results using TF-IDF:")
print("Reciprocal Rank:", mean_rr_tf_idf)
print("Top-3 Accuracy:", mean_top3_tf_idf)

print("Results using Bhattacharyya:")
print("Reciprocal Rank:", mean_rr_bhattacharyya)
print("Top-3 Accuracy:", mean_top3_bhattacharyya)

print("Results using Kullback-Leibler:")
print("Reciprocal Rank:", mean_rr_kullback_leibler)
print("Top-3 Accuracy:", mean_top3_kullback_leibler)

Results using Common Words:
Reciprocal Rank: 0.2585176273342592
Top-3 Accuracy: 0.2889447236180904
Results using TF-IDF:
Reciprocal Rank: 0.6358200411516305
Top-3 Accuracy: 0.6943048576214406
Results using Bhattacharyya:
Reciprocal Rank: 0.5793823117209481
Top-3 Accuracy: 0.6566164154103853
Results using Kullback-Leibler:
Reciprocal Rank: 0.38715323206838864
Top-3 Accuracy: 0.5226130653266332
